In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style

In [7]:
students = pd.read_csv("students.csv")
methods = pd.read_csv("methods.csv")
modules = pd.read_csv("modules.csv")
teams = pd.read_csv("modules.csv")
indAggResponses = pd.read_csv("indAggResponses.csv")
teamAggResponses = pd.read_csv("teamAggResponses.csv")

In [10]:
teams

,Unnamed: 0,Module,Module #
0,MOD_0,Research,1
1,MOD_1,Analysis,2
2,MOD_2,Ideate,3
3,MOD_3,Build,4
4,MOD_4,Communicate,5
